In [1]:
'''
Data used here is from Northern Hemisphere EASE-Grid 2.0 Weekly Snow Cover and Sea Ice Extent, Version 4

''' 

# Modules used in this program

%matplotlib qt
import yaml
yaml.warnings({'YAMLLoadWarning': False})
# the default loader is deprecated, I dont know how to change the default loader
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import random
import math
from scipy import stats
import seaborn as sns 
#random.seed(1234)

In [2]:
# Reading the netcdf file

whole_dataset = r"C:\Users\krish\Desktop\Courses\Thesis\data\downloadtrials\snow\complete.nc"
dataset = xr.open_dataset(whole_dataset, chunks = {'time':10})#, 'row':10, 'col':10})

In [3]:
# Attributes of netcdf file

dataset

<xarray.Dataset>
Dimensions:     (col: 720, row: 720, time: 2689)
Coordinates:
  * row         (row) int32 0 1 2 3 4 5 6 7 ... 712 713 714 715 716 717 718 719
  * col         (col) int32 0 1 2 3 4 5 6 7 ... 712 713 714 715 716 717 718 719
  * time        (time) datetime64[ns] 1966-10-03 1966-10-10 ... 2018-12-24
Data variables:
    snow_cover  (time, row, col) int8 dask.array<shape=(2689, 720, 720), chunksize=(10, 720, 720)>
    lats        (row, col) float64 dask.array<shape=(720, 720), chunksize=(720, 720)>
    lons        (row, col) float64 dask.array<shape=(720, 720), chunksize=(720, 720)>

In [4]:
# Narrowing the dataset to spring values

springs = []
for year in np.arange(1979,2019):
    beginning = "%d-03-01" %(year)
    end = "%d-08-30" %(year)
    dataset_spring = dataset.sel(time=slice(beginning,end))
    
    # Dataset snowcover values resampled from Weekly data to Monthly data here
    springs.append(dataset_spring.resample(time='M').mean())
    


all_springs = xr.concat(springs, 'time', data_vars = 'minimal')

In [5]:
# Description of the processed spring snowcover data. 
all_springs

<xarray.Dataset>
Dimensions:     (col: 720, row: 720, time: 240)
Coordinates:
  * row         (row) int32 0 1 2 3 4 5 6 7 ... 712 713 714 715 716 717 718 719
  * col         (col) int32 0 1 2 3 4 5 6 7 ... 712 713 714 715 716 717 718 719
  * time        (time) datetime64[ns] 1979-03-31 1979-04-30 ... 2018-08-31
Data variables:
    snow_cover  (time, row, col) float64 dask.array<shape=(240, 720, 720), chunksize=(1, 720, 720)>
    lats        (time, row, col) float64 dask.array<shape=(240, 720, 720), chunksize=(1, 720, 720)>
    lons        (time, row, col) float64 dask.array<shape=(240, 720, 720), chunksize=(1, 720, 720)>

In [6]:
# Climatology plots

March, April, May = np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720)) 
June, July, August = np.zeros((720,720)), np.zeros((720,720)), np.zeros((720,720))
for i in np.arange(0,40):
    
    March += all_springs.snow_cover[0+i*6,:,:].values
    April += all_springs.snow_cover[1+i*6,:,:].values
    May += all_springs.snow_cover[2+i*6,:,:].values
    June += all_springs.snow_cover[3+i*6,:,:].values
    July += all_springs.snow_cover[4+i*6,:,:].values
    August += all_springs.snow_cover[5+i*6,:,:].values



In [7]:
# Calculating the Mean for each month

March = March/40
April = April/40
May = May/40
June = June/40
July = July/40
August = August/40


In [8]:
# Plotting the Climatology Maps

plt.figure()
plt.title("Climatology map for March (1979-2018)")
plt.imshow(March, origin = 'lower', cmap = 'binary_r')
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Climatology map for April (1979-2018)")
plt.imshow(April, origin = 'lower', cmap = 'binary_r')
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Climatology map for May (1979-2018)")
plt.imshow(May, origin = 'lower', cmap = 'binary_r')
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Climatology map for June (1979-2018)")
plt.imshow(June, origin = 'lower', cmap = 'binary_r')
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Climatology map for July (1979-2018)")
plt.imshow(July, origin = 'lower', cmap = 'binary_r')
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Climatology map for August (1979-2018)")
plt.imshow(August, origin = 'lower', cmap = 'binary_r')
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.show()

In [ ]:
# Trend map


years = np.arange(0, 40) 
slopes_march = np.empty([720, 720])
intercepts_march = np.empty([720, 720])
slopes_april = np.empty([720, 720])
intercepts_april = np.empty([720, 720])
slopes_may = np.empty([720, 720])
intercepts_may = np.empty([720, 720])
slopes_june = np.empty([720, 720])
intercepts_june = np.empty([720, 720])
slopes_july = np.empty([720, 720])
intercepts_july = np.empty([720, 720])
slopes_august = np.empty([720, 720])
intercepts_august = np.empty([720, 720])

for row in np.arange(720):
    march_values = []
    april_values = []
    may_values = []
    june_values = []
    july_values = []
    august_values = []
    w_array_march = all_springs.snow_cover[0:240:6,row,:].values
    w_array_april = all_springs.snow_cover[1:240:6,row,:].values
    w_array_may = all_springs.snow_cover[2:240:6,row,:].values
    w_array_june = all_springs.snow_cover[3:240:6,row,:].values
    w_array_july = all_springs.snow_cover[4:240:6,row,:].values
    w_array_august = all_springs.snow_cover[5:240:6,row,:].values
    for col in np.arange(720):
        
        march_values.append(w_array_march[:,col])
        april_values.append(w_array_april[:,col])
        may_values.append(w_array_may[:,col])
        june_values.append(w_array_june[:,col])
        july_values.append(w_array_july[:,col])
        august_values.append(w_array_august[:,col])
        
        m, b = np.polyfit(years, march_values[col], 1)
        slopes_march[row, col] = m
        intercepts_march[row, col] = b
        
        m, b = np.polyfit(years, april_values[col], 1)
        slopes_april[row, col] = m
        intercepts_april[row, col] = b
        
        m, b = np.polyfit(years, may_values[col], 1)
        slopes_may[row, col] = m
        intercepts_may[row, col] = b
        
        m, b = np.polyfit(years, june_values[col], 1)
        slopes_june[row, col] = m
        intercepts_june[row, col] = b
        
        m, b = np.polyfit(years, july_values[col], 1)
        slopes_july[row, col] = m
        intercepts_july[row, col] = b
        
        m, b = np.polyfit(years, august_values[col], 1)
        slopes_august[row, col] = m
        intercepts_august[row, col] = b
        

In [ ]:
# Plots of trend maps
color = 'seismic'
plt.figure()    
plt.title("Trend map for March (1979-2018)")
#cmap = colors.ListedColormap(['black', 'white'])
plt.imshow(slopes_march*40, vmax = 1, vmin = -1, origin = 'lower', cmap = color)
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Trend map for April (1979-2018)")
plt.imshow(slopes_april*40, vmax = 1, vmin = -1, origin = 'lower', cmap = color)
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Trend map for May (1979-2018)")
plt.imshow(slopes_may*40, vmax = 1, vmin = -1, origin = 'lower', cmap = color)
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Trend map for June (1979-2018)")
plt.imshow(slopes_june*40, vmax = 1, vmin = -1, origin = 'lower', cmap = color)
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Trend map for July (1979-2018)")
plt.imshow(slopes_july*40, vmax = 1, vmin = -1, origin = 'lower', cmap = color)
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Trend map for August (1979-2018)")
plt.imshow(slopes_august*40, vmax = 1, vmin = -1, origin = 'lower', cmap = color)
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.show()

In [ ]:
# Anomaly maps

#Variables storing values of 1st 20 years of the dataset
March_b, April_b, May_b, June_b, July_b, August_b = np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720])
#Variables storing values of last 20 years of the dataset
March_l, April_l, May_l, June_l, July_l, August_l = np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720]), np.zeros([720,720])


for i in np.arange(0,40):
    
    if i < 20:
        March_b += all_springs.snow_cover[0+i*6,:,:].values
        April_b += all_springs.snow_cover[1+i*6,:,:].values
        May_b += all_springs.snow_cover[2+i*6,:,:].values
        June_b += all_springs.snow_cover[3+i*6,:,:].values
        July_b += all_springs.snow_cover[4+i*6,:,:].values
        August_b += all_springs.snow_cover[5+i*6,:,:].values
    else:
        March_l += all_springs.snow_cover[0+i*6,:,:].values
        April_l += all_springs.snow_cover[1+i*6,:,:].values
        May_l += all_springs.snow_cover[2+i*6,:,:].values
        June_l += all_springs.snow_cover[3+i*6,:,:].values
        July_l += all_springs.snow_cover[4+i*6,:,:].values
        August_l += all_springs.snow_cover[5+i*6,:,:].values
        
anomaly_march = (March_l - March_b)/20
anomaly_april = (April_l - April_b)/20
anomaly_may = (May_l - May_b)/20
anomaly_june = (June_l - June_b)/20
anomaly_july = (July_l - July_b)/20
anomaly_august = (August_l - August_b)/20

In [ ]:
# Plots of Anomaly maps

plt.figure()    
color = 'seismic'
plt.title("Anomaly map for March (1979-2018)")

plt.imshow(anomaly_march, vmax = 1, vmin = -1, origin = 'lower', cmap = color)
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Anomaly map for April (1979-2018)")
plt.imshow(anomaly_april, vmax = 1, vmin = -1, origin = 'lower', cmap = color)
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Anomaly map for May (1979-2018)")
plt.imshow(anomaly_may, vmax = 1, vmin = -1, origin = 'lower', cmap = color)
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Anomaly map for June (1979-2018)")
plt.imshow(anomaly_june, vmax = 1, vmin = -1, origin = 'lower', cmap = color)
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Anomaly map for July (1979-2018)")
plt.imshow(anomaly_july, vmax = 1, vmin = -1, origin = 'lower', cmap = color)
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.figure()
plt.title("Anomaly map for August (1979-2018)")
plt.imshow(anomaly_august, vmax = 1, vmin = -1, origin = 'lower', cmap = color)
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)

plt.show()

In [ ]:
def find_average(snowcover_data):
    """
    Returns the spatial average snowcover and spatial aggregate snowcover for a particular month
    """
    total = 0
    for i in np.arange(720):
        for j in np.arange(720):
            total += snowcover_data[i,j]
    average = total/(720*720)
    #total is multiplied with 625 for spatial aggregate as the area of each grid cell is 25km*25km
    return average, total*625

In [ ]:
# Trend lines


coverage_march = []
coverage_april = []
coverage_may = []
coverage_june = []
coverage_july = []
coverage_august = []

average_march = find_average(March)
average_april = find_average(April)
average_may = find_average(May)
average_june = find_average(June)
average_july = find_average(July)
average_august = find_average(August)

for i in np.arange(0,40):
    #mar = '%d-03-31'%(year)
    coverage_march.append(find_average(all_springs.snow_cover[0+i*6,:,:].values))
    coverage_april.append(find_average(all_springs.snow_cover[1+i*6,:,:].values))
    coverage_may.append(find_average(all_springs.snow_cover[2+i*6,:,:].values))
    coverage_june.append(find_average(all_springs.snow_cover[3+i*6,:,:].values))
    coverage_july.append(find_average(all_springs.snow_cover[4+i*6,:,:].values))
    coverage_august.append(find_average(all_springs.snow_cover[5+i*6,:,:].values)) 
#print(average_march, average_april)

In [ ]:
#plot of trend lines:
years = np.arange(1979, 2019)
plt.figure()
plt.title("Trend of Snow cover -NH- March 1979-2018")

plt.plot(years, [(coverage-average_march)/average_march for coverage in coverage_march])
plt.plot(years, [0]*len(years), '--')

plt.xlabel("years")
plt.ylabel("Relative snowocover average snowcover(1979-2018)")
"""
plt.figure()
plt.title("Average snow cover -NH- March 1979-2018")

plt.plot(years, coverage_april)
plt.plot(years, [average_april]*len(years), '--')
plt.xlabel("years")
plt.ylabel("amount of area covered")
plt.figure()
plt.title("Average snow cover -NH- March 1979-2018")

plt.plot(years, coverage_may)
plt.plot(years, [average_may]*len(years), '--')
plt.xlabel("years")
plt.ylabel("amount of area covered")
plt.figure()
plt.title("Average snow cover -NH- March 1979-2018")

plt.plot(years, coverage_june)
plt.plot(years, [average_june]*len(years), '--')
plt.xlabel("years")
plt.ylabel("amount of area covered")
plt.figure()
plt.title("Average snow cover -NH- March 1979-2018")

plt.plot(years, coverage_july)
plt.plot(years, [average_july]*len(years), '--')
plt.xlabel("years")
plt.ylabel("amount of area covered")
plt.figure()
plt.title("Average snow cover -NH- March 1979-2018")

plt.plot(years, coverage_august)
plt.plot(years, [average_august]*len(years), '--')
plt.xlabel("years")
plt.ylabel("amount of area covered")
"""
plt.show()